In [9]:
import pandas as pd
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler
from joblib import dump

In [3]:
df_data = pd.read_csv("data_separated/train_data.csv")
df_data

,age_at_diagnosis,chemotherapy,cohort,neoplasm_histologic_grade,hormone_therapy,lymph_nodes_examined_positive,mutation_count,nottingham_prognostic_index,overall_survival_months,radio_therapy,...,srd5a1,srd5a2,srd5a3,st7,star,tnk2,tulp4,ugt2b15,ugt2b17,ugt2b7
0,61.88,0,3.0,3.0,1,0.0,7.0,4.070,77.466667,0,...,-0.3913,-0.2902,0.3308,-1.1936,-0.3610,0.9515,-0.6489,-0.7160,-0.9414,-0.5863
1,43.39,1,1.0,3.0,1,2.0,2.0,5.040,131.066667,1,...,-0.7080,-1.3323,0.5152,-0.1132,0.5610,-0.8075,-0.4297,0.9704,-0.0399,-0.8509
2,59.07,1,1.0,3.0,1,3.0,5.0,5.070,143.133333,1,...,-0.2810,-0.3956,-1.5129,0.7664,-0.6735,0.2051,0.0839,0.0337,-0.4190,-0.1072
3,84.22,0,1.0,2.0,0,0.0,5.0,3.056,36.266667,0,...,-0.3020,-0.4788,3.6125,1.9925,0.2253,-0.8414,1.7093,-0.6921,0.6906,-0.8006
4,77.94,0,1.0,1.0,1,0.0,5.0,2.094,151.166667,0,...,0.7152,0.1752,-0.2471,0.1327,-0.6543,0.5775,-0.2020,-0.4854,-0.0629,-0.3033
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
691,37.30,1,1.0,2.0,1,1.0,6.0,4.056,72.666667,0,...,-0.5509,0.1183,-0.7828,0.1648,-0.1488,0.0954,-0.2825,0.6950,0.2156,-0.7896
692,57.96,0,2.0,2.0,0,0.0,3.0,3.040,59.700000,1,...,1.0545,1.2233,0.6252,-0.2022,-0.8374,-0.8369,-0.2616,-0.7472,-0.4232,-0.1055
693,41.53,1,3.0,3.0,0,4.0,3.0,6.040,180.633333,1,...,1.5865,-0.6735,1.2946,1.4392,-0.3016,-0.0598,-1.6409,-0.1382,0.3853,-0.7615
694,38.01,0,5.0,2.0,1,0.0,2.0,3.036,193.966667,0,...,-0.9936,0.2779,0.3695,-0.7440,-1.0271,-0.1082,-0.5885,0.0373,-0.3350,-0.5733


In [12]:
std_scaler_instance = StandardScaler()
std_scaler_instance.fit(X=df_data.values)
data_scaled_std = std_scaler_instance.transform(X=df_data.values)

In [13]:
minmax_scaler_instance = MinMaxScaler()
minmax_scaler_instance.fit(X=df_data.values)
data_scaled_minmax = minmax_scaler_instance.transform(X=df_data.values)

In [14]:
robust_scaler_instance = RobustScaler()
robust_scaler_instance.fit(X=df_data.values)
data_scaled_robust = robust_scaler_instance.transform(X=df_data.values)

In [17]:
df_scaled_std = pd.DataFrame(data=data_scaled_std, columns=df_data.columns)
df_scaled_minmax = pd.DataFrame(data=data_scaled_minmax, columns=df_data.columns)
df_scaled_robust = pd.DataFrame(data=data_scaled_robust, columns=df_data.columns)

In [23]:
df_scaled_std.to_csv("data_separated/train_data_std.csv", index=False)
df_scaled_minmax.to_csv("data_separated/train_data_minmax.csv", index=False)
df_scaled_robust.to_csv("data_separated/train_data_robust.csv", index=False)

In [22]:
dump(std_scaler_instance, "results/scaler_std.joblib")
dump(minmax_scaler_instance, "results/scaler_std.joblib")
dump(robust_scaler_instance, "results/scaler_std.joblib")

['results/scaler_std.joblib']